# Checkpoint 4 - Visão Computacional

---

## Aluno
> Henrico Nardelli Bela | 
> Emilly Gabrielly | 
> Felype Nunes | 
> Sara Leal | 
> Daniel Faria | 
## RM
> RM 95985 | 
> RM 94437 | 
> RM 96232 | 
> RM 96302 | 
> RM 94026 | 
## Turma
> 2TIAR


---

In [31]:
!pip install transformers datasets evaluate tensorflow keras torch

In [2]:
#importações necessárias
from datasets import load_dataset
import numpy as np
from PIL import Image
import evaluate
from transformers import DefaultDataCollator
from transformers import create_optimizer
from transformers import TFAutoModelForImageClassification
from transformers import pipeline
from transformers import AutoImageProcessor
from transformers import TFAutoModelForImageClassification
from transformers import AutoImageProcessor
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow import keras
from tensorflow.keras import layers



In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# Converte uma imagem em um tensor TensorFlow.
def convert_to_tf_tensor(image: Image):
    np_image = np.array(image)
    tf_image = tf.convert_to_tensor(np_image)
    return tf.expand_dims(tf_image, 0)

# Aplica transformações de dados de treinamento em um lote de exemplos.
def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    images = [
        train_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

#Aplica transformações de dados de validação em um lote de exemplos.
def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    images = [
        val_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

#Calcula métricas de avaliação com base nas previsões e rótulos fornecidos.
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [4]:
# Carrega o conjunto de dados "food101" e seleciona os primeiros 5000 exemplos da divisão de treinamento.
food = load_dataset("food101", split="train[:5000]")
# Divide o conjunto de dados em treinamento e teste, reservando 20% dos dados para o conjunto de teste.
food = food.train_test_split(test_size=0.2)

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [5]:
# Acessa o primeiro exemplo (amostra) do conjunto de treinamento no conjunto de dados "food".
food["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 'label': 81}

In [6]:
# Obtém os nomes das categorias de rótulos presentes no conjunto de treinamento.
labels = food["train"].features["label"].names

# Cria dicionários para mapear nomes de categorias de rótulos para identificadores e vice-versa.
label2id, id2label = dict(), dict()

# Itera sobre as categorias de rótulos e cria mapeamentos.
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Obtém o nome da categoria de rótulo correspondente ao identificador 79 usando o dicionário id2label.
label_79 = id2label[str(79)]

In [7]:
#especifica qual o modelo pré-treinado
checkpoint = "google/vit-base-patch16-224-in21k"
# Cria uma instância de AutoImageProcessor a partir do modelo pré-treinado especificado.
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [9]:
# Obtém as dimensões da imagem processada a partir do image_processor.
size = (image_processor.size["height"], image_processor.size["width"])

# Define um pipeline de aumentos de dados para o conjunto de treinamento.
train_data_augmentation = keras.Sequential(
    [
        layers.RandomCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="train_data_augmentation",  # Nomeie a sequência de camadas.
)

# Define um pipeline de aumentos de dados para o conjunto de validação.
val_data_augmentation = keras.Sequential(
    [
        layers.CenterCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
    ],
    name="val_data_augmentation",
)


In [10]:
# Aplica a função de pré-processamento "preprocess_train" ao conjunto de treinamento.
food["train"].set_transform(preprocess_train)

# Aplica a função de pré-processamento "preprocess_val" ao conjunto de teste.
food["test"].set_transform(preprocess_val)


In [11]:
# Cria um objeto DataCollator padrão para processamento de dados.
data_collator = DefaultDataCollator(return_tensors="tf")

In [12]:
# carrega a metrica de avaliação acuracia
accuracy = evaluate.load("accuracy")

In [13]:
# Define o tamanho do lote (batch size) a ser usado no treinamento.
batch_size = 16

# Define o número de épocas (epochs) para treinamento.
num_epochs = 5

# Calcula o número total de etapas de treinamento com base no número de exemplos e épocas.
num_train_steps = len(food["train"]) * num_epochs

# Define a taxa de aprendizado inicial.
learning_rate = 3e-5

# Define a taxa de decaimento de peso (weight decay rate) para regularização.
weight_decay_rate = 0.01

# Cria um otimizador e um agendamento de taxa de aprendizado com base nos parâmetros especificados.
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0,
)


In [14]:
# Cria uma instância do modelo de classificação de imagem TFAutoModelForImageClassification a partir de um modelo pré-treinado.
model = TFAutoModelForImageClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)


Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/bias:0', 'vit/pooler/dense/kernel:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Converte o conjunto de treinamento "food['train']" em um conjunto de dados TensorFlow (tf.data.Dataset).
tf_train_dataset = food["train"].to_tf_dataset(
    columns="pixel_values",
    label_cols="label",
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)

# Converte o conjunto de teste "food['test']" em um conjunto de dados TensorFlow (tf.data.Dataset).
tf_eval_dataset = food["test"].to_tf_dataset(
    columns="pixel_values",  # Especifica as colunas (características) que serão usadas como entrada.
    label_cols="label",  # Especifica as colunas que contêm os rótulos das classes.
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)


In [16]:
# Define a função de perda como a entropia cruzada categórica esparsa.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compila o modelo, especificando o otimizador e a função de perda.
model.compile(optimizer=optimizer, loss=loss)

In [19]:
# Cria um callback para métricas personalizadas durante a avaliação.
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_eval_dataset)

# Cria um callback para fazer push do modelo treinado para um repositório no Hugging Face Hub.
push_to_hub_callback = PushToHubCallback(
    output_dir="food_classifier",
    tokenizer=image_processor,
    save_strategy="no",
)

# Cria uma lista de callbacks que serão usados durante o treinamento.
callbacks = [metric_callback, push_to_hub_callback]


Cloning https://huggingface.co/henrico219/food_classifier into local empty directory.


In [20]:
# Treina o modelo usando o conjunto de treinamento, com validação no conjunto de teste, e aplica os callbacks.
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5
250/250 [==============================] - 390s 1s/step - loss: 2.7731 - val_loss: 1.5928 - accuracy: 0.8300
Epoch 2/5
250/250 [==============================] - 371s 1s/step - loss: 1.2255 - val_loss: 0.8033 - accuracy: 0.9030
Epoch 3/5
250/250 [==============================] - 357s 1s/step - loss: 0.7124 - val_loss: 0.5400 - accuracy: 0.9060
Epoch 4/5
250/250 [==============================] - 375s 2s/step - loss: 0.5017 - val_loss: 0.4041 - accuracy: 0.9110
Epoch 5/5
250/250 [==============================] - 364s 1s/step - loss: 0.3847 - val_loss: 0.3365 - accuracy: 0.9260


Upload file tf_model.h5:   0%|          | 1.00/328M [00:00<?, ?B/s]

To https://huggingface.co/henrico219/food_classifier
   59cd61f..9caaf36  main -> main

   59cd61f..9caaf36  main -> main



In [21]:
# Carrega um subconjunto do conjunto de dados "food101" para validação, contendo apenas as 10 primeiras amostras.
ds = load_dataset("food101", split="validation[:10]")

# Recupera a primeira imagem do subconjunto.
image = ds["image"][0]


In [34]:
#Cria uma instância de um pipeline de classificação de imagem chamado "classifier" com base no modelo "my_awesome_food_model".
classifier = pipeline("image-classification", model="food_classifier")
# Realiza uma classificação de imagem usando o modelo "classifier" na imagem "image".
classifier(image)

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at food_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


[{'score': 0.9174814224243164, 'label': 'beignets'},
 {'score': 0.0071230498142540455, 'label': 'bruschetta'},
 {'score': 0.005839253775775433, 'label': 'prime_rib'},
 {'score': 0.0057570175267755985, 'label': 'pork_chop'},
 {'score': 0.005420571658760309, 'label': 'chicken_wings'}]

In [35]:
# Carrega um processador de imagem automático a partir de um modelo pré-treinado chamado "MariaK/food_classifier".
image_processor = AutoImageProcessor.from_pretrained("food_classifier")

# Processa a imagem de entrada utilizando o processador de imagem e retorna os tensores no formato TensorFlow.
inputs = image_processor(image, return_tensors="tf")

In [37]:
# Carrega um modelo pré-treinado de classificação de imagem chamado "MariaK/food_classifier".
model = TFAutoModelForImageClassification.from_pretrained("food_classifier")

# Realiza a inferência do modelo nas entradas processadas e obtém as saídas (logits) do modelo.
logits = model(**inputs).logits


All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at food_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [38]:
# Calcula o ID da classe prevista com base nos logits (valores de saída do modelo).
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])

# Mapeia o ID da classe prevista para o nome da classe usando a configuração do modelo.
predicted_class_name = model.config.id2label[predicted_class_id]


In [39]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'beignets'

---

# Relatório de Classificação de Imagens com o Modelo Vision Transformer (ViT)

## 1. Introdução

Este relatório descreve o processo de classificação de imagens usando o modelo Vision Transformer (ViT) e suas implementações no framework Hugging Face Transformers e TensorFlow. O código implementa o treinamento de um modelo ViT para classificar imagens em um conjunto de dados de alimentos ("food101").

## 2. Importações e Configurações

O código começa importando as bibliotecas necessárias, incluindo as implementações dos modelos ViT da Hugging Face, TensorFlow e outras bibliotecas de processamento de imagem e avaliação.

## 3. Pré-processamento de Imagens

O código define funções de pré-processamento de imagens para o treinamento e validação. Essas funções aplicam transformações de dados, como recorte aleatório, rotação, espelhamento e redimensionamento, para aumentar a variabilidade dos dados de treinamento.

## 4. Carregamento e Divisão de Dados

O conjunto de dados "food101" é carregado e dividido em conjuntos de treinamento e teste. 20% dos dados são reservados para o conjunto de teste.

## 5. Configurações do Modelo

- O código define o processador de imagens (image_processor) com base em um modelo pré-treinado (checkpoint).
- Configura o tamanho das imagens com base nas configurações do processador.
- Define o otimizador e programação de taxa de aprendizado (learning rate schedule).

## 6. Treinamento do Modelo

- Carrega um modelo ViT para classificação de imagens com base no checkpoint.
- Configura o treinamento do modelo, incluindo a escolha da função de perda (cross-entropia categórica esparsa) e compila o modelo com o otimizador.
- Converte o conjunto de treinamento e teste em objetos `tf.data.Dataset`.
- Treina o modelo usando o conjunto de treinamento e monitora o desempenho usando métricas como a precisão (accuracy).
- Implementa callbacks, como a métrica de avaliação e o envio para o Hugging Face Model Hub.

## 7. Classificação de Imagens

- Carrega uma imagem de exemplo do conjunto de validação.
- Cria um pipeline para classificação de imagens usando o modelo treinado.
- Classifica a imagem de exemplo e retorna a classe prevista.

## 8. Conclusão

O código implementa com sucesso o treinamento de um modelo Vision Transformer (ViT) para classificação de imagens em um conjunto de dados de alimentos. Ele demonstra o uso das bibliotecas Hugging Face Transformers e TensorFlow para realizar tarefas de visão computacional. A precisão do modelo e a capacidade de classificar imagens de alimentos são tópicos de interesse.